In [4]:
import numpy as np
import torch

In [5]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [6]:
# Convert inputs and targets to tensors
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [7]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[-0.8675, -0.4186,  0.4995],
        [ 1.0118,  0.8942,  0.0157]], requires_grad=True)
tensor([-0.2413,  0.0188], requires_grad=True)


In [8]:
# Linear regression 
def model(x):
    return x @ w.t() + b

In [9]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[ -70.1350,  134.4650],
        [ -84.0505,  171.7852],
        [-102.8311,  208.7761],
        [ -88.2448,  142.2516],
        [ -65.3164,  156.7741]], grad_fn=<AddBackward0>)


In [10]:
# Creating a lost function
def mse(t1,t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel()

In [11]:
# Compute loss
loss = mse(preds, targets)
print(loss)

tensor(16025.8174, grad_fn=<DivBackward0>)


### Compute Gradients

In [12]:
loss.backward()

In [13]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[-0.8675, -0.4186,  0.4995],
        [ 1.0118,  0.8942,  0.0157]], requires_grad=True)
tensor([[-13277.1123, -14719.9531,  -8942.8906],
        [  6216.3975,   5770.8750,   3647.1506]])


### Training

In [14]:
# we will train for 200 epochs
for epoch in range(300):
    preds = model(inputs) # prediction 
    loss = mse(preds, targets) # calculate loss
    loss.backward() # calculate gradients
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
    if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

Epoch [10/300], Loss: 680.7848
Epoch [20/300], Loss: 371.2711
Epoch [30/300], Loss: 328.9848
Epoch [40/300], Loss: 296.0746
Epoch [50/300], Loss: 267.0598
Epoch [60/300], Loss: 241.3932
Epoch [70/300], Loss: 218.6761
Epoch [80/300], Loss: 198.5583
Epoch [90/300], Loss: 180.7319
Epoch [100/300], Loss: 164.9252
Epoch [110/300], Loss: 150.8996
Epoch [120/300], Loss: 138.4440
Epoch [130/300], Loss: 127.3732
Epoch [140/300], Loss: 117.5239
Epoch [150/300], Loss: 108.7521
Epoch [160/300], Loss: 100.9309
Epoch [170/300], Loss: 93.9489
Epoch [180/300], Loss: 87.7074
Epoch [190/300], Loss: 82.1200
Epoch [200/300], Loss: 77.1103
Epoch [210/300], Loss: 72.6108
Epoch [220/300], Loss: 68.5624
Epoch [230/300], Loss: 64.9128
Epoch [240/300], Loss: 61.6161
Epoch [250/300], Loss: 58.6314
Epoch [260/300], Loss: 55.9234
Epoch [270/300], Loss: 53.4602
Epoch [280/300], Loss: 51.2141
Epoch [290/300], Loss: 49.1608
Epoch [300/300], Loss: 47.2786


In [15]:
preds = model(inputs)
print(preds)

tensor([[ 57.4594,  72.1627],
        [ 84.6464,  95.7196],
        [112.6711, 141.2377],
        [ 23.2462,  47.1527],
        [104.8547, 104.6119]], grad_fn=<AddBackward0>)


## Linear regression using PyTorch built-ins

In [16]:
import torch.nn as nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F

In [17]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [18]:
# Define dataset
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [19]:
# Define model
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.5400, -0.4189, -0.3970],
        [-0.4150,  0.2960,  0.0733]], requires_grad=True)
Parameter containing:
tensor([0.1484, 0.0091], requires_grad=True)


In [20]:
loss_fn = F.mse_loss

In [21]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [22]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

In [23]:
fit(100, model, loss_fn, opt, train_dl)

# Print preds
preds = model(inputs)
preds

Epoch [10/100], Loss: 375.4182
Epoch [20/100], Loss: 656.2269
Epoch [30/100], Loss: 244.0938
Epoch [40/100], Loss: 174.5236
Epoch [50/100], Loss: 121.5028
Epoch [60/100], Loss: 169.4109
Epoch [70/100], Loss: 89.4336
Epoch [80/100], Loss: 125.8663
Epoch [90/100], Loss: 55.6358
Epoch [100/100], Loss: 44.7388


tensor([[ 59.1582,  71.0038],
        [ 79.5849,  98.0566],
        [119.8647, 137.3406],
        [ 33.3838,  41.0827],
        [ 90.3237, 112.2524],
        [ 58.1538,  69.9246],
        [ 78.8775,  97.6759],
        [119.9315, 137.6965],
        [ 34.3882,  42.1619],
        [ 90.6208, 112.9509],
        [ 58.4508,  70.6231],
        [ 78.5804,  96.9774],
        [120.5721, 137.7213],
        [ 33.0867,  40.3841],
        [ 91.3282, 113.3316]], grad_fn=<AddmmBackward0>)